### Entanglement-based Quantum Key Distribution in a $n$ x $n$ grid topology using Netsquid

#### The objetive of this project is to use the Ekert Protocol in order to implement QKD in a $n$ x $n$ Quantum Network. The following list describes the milestones to be achieved:
* Entangle a random route using Entanglement Swapping (Completed)
* Entangle two or more disjunctive routes (Completed)
* Generate the shared key in each node (In process)  

#### Code functionality: This implementation creates a $n$ x $n$ grid Quantum Network. After the Quantum Network is defined, the simulation starts dynamic protocols depending on the connection requirements. When the simulation ends, it brings back the path, fidelity and time information.

_refs_: 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater.html 
* Evan Sutcliffe, Matty J. Hoban & Alejandra Beghelli - Multipath Routing for Multipartite State Distribution in Quantum Networks  

_(adapted by D-Cryp7 for Netsquid 1.1.6)_

#### Limitations:
* Conjunctive routes causes a Race Condition error. For now, our aim focus on disjunctive routes, due to the possible alternative options.

In [1]:
# Library imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from custom_netsquid_functions import *
from base_netsquid_functions import *
from util import *

In [ ]:
def setup_datacollector(network, protocol, path, node_qconnections):
    """
    Setup the datacollector to calculate the fidelity
    when the CorrectionProtocol has finished.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    protocol : :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    Returns
    -------
    :class:`~netsquid.util.datacollector.DataCollector`
        Datacollector recording fidelity data.

    """

    # Ensure nodes are ordered in the chain:
    # nodes = [network.nodes[name] for name in sorted(network.nodes.keys())]
    nodes = [network.nodes[str(name)] for name in path]
    A_port = int(node_qconnections[nodes[0].name][nodes[1].name].split("-")[0][-1])
    B_port = int(node_qconnections[nodes[-1].name][nodes[-2].name].split("-")[0][-1])
    
    def calc_fidelity(evexpr):
        qubit_a, = nodes[0].qmemory.peek([A_port])
        qubit_b, = nodes[-1].qmemory.peek([B_port])
        fidelity = ns.qubits.fidelity([qubit_a, qubit_b], ks.b00, squared = True)
        return {"fidelity": fidelity}

    dc = DataCollector(calc_fidelity, include_entity_name=False)
    dc.collect_on(pydynaa.EventExpression(source=protocol.subprotocols['CorrectProtocol'],
                                          event_type=Signals.SUCCESS.value))
    return dc

#### Record Fidelity and Loss metrics (PoC)

In [ ]:
%%time

import time

# Default values

n = 10
node_distance = 20
num_iters = 256
est_runtime = (0.5 + 2 * n - 1) * node_distance * 5e3

results = {
    "path": [],
    "path_length": [],
    "fidelity_rate": [],
    "loss": []
}

try:
    file = open(f"results_{n}x{n}.csv", "r").close()
    print(f"Found results_{n}x{n}.csv")
except:
    print(f"Creating results_{n}x{n}.csv")
    file = open(f"results_{n}x{n}.csv", "w").write("path,path_length,fidelity_rate,loss")

for i in range(10000):
    ns.sim_reset()
    network, node_qconnections = network_setup(n, node_distance = node_distance,
                            source_frequency = 1e9 / est_runtime)

    qdf = pandas.DataFrame(node_qconnections)
    qdf = qdf.fillna("")
    
    path = get_random_route(network)

    traffic = {
        "path": [path]
    }

    # print(f"Simulation from path: {path}")

    df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)

    rate = df[0].dataframe["fidelity"].sum() / num_iters
    loss = 1 - (df[0].dataframe.shape[0] / num_iters)

    results_df = pandas.read_csv(f"results_{n}x{n}.csv")
    results_df.loc[len(results_df.index)] = [path, len(path), rate * 100, loss * 100]
    results_df.to_csv(f"results_{n}x{n}.csv", index = False)

#### Mean attemps until successful entanglement

In [ ]:
%%time

import time

# Default values

n = 3
length = 3 # length of path
node_distance = 20
num_iters = 256
est_runtime = (0.5 + 2 * n - 1) * node_distance * 5e3

results = {
    "fidelity": [],
    "mean attemps": []
}

try:
    file = open(f"entanglement_probability_results_{n}x{n}_length_{length}.csv", "r").close()
    print(f"entanglement_probability_results_{n}x{n}_length_{length}.csv exists")
except:
    print(f"Creating entanglement_probability_results_{n}x{n}_length_{length}.csv")
    file = open(f"entanglement_probability_results_{n}x{n}_length_{length}.csv", "w").write("fidelity")

for i in range(10000):
    ns.sim_reset()
    network, node_qconnections = network_setup(n, node_distance = node_distance,
                            source_frequency = 1e9 / est_runtime)

    qdf = pandas.DataFrame(node_qconnections)
    qdf = qdf.fillna("")
    
    path = get_random_route(network)
    while len(path) != length:
        path = get_random_route(network)

    traffic = {
        "path": [path]
    }

    # print(f"Simulation from path: {path}")

    df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)
    
    results_df = pandas.read_csv(f"entanglement_probability_results_{n}x{n}_length_{length}.csv")
    for i in df[0].dataframe["fidelity"].tolist():
        fidelity_results = results_df["fidelity"].tolist()
        runs = zero_runs(fidelity_results) # sequences of zeroes in the fidelity list
        if len(runs) == 0:
            runs = 0
        else:
            runs = sum(runs[:,1] - runs[:,0]) / len(runs)
        results_df = results_df.append({'fidelity': i, "mean attemps": runs}, ignore_index=True)
        
    results_df.to_csv(f"entanglement_probability_results_{n}x{n}_length_{length}.csv", index = False)